In [1]:
import pandas as pd
import altair as alt
from functools import reduce
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [2]:
# data selection
df_raw = pd.read_csv('data.csv')
df = df_raw.copy()

In [3]:
# data translation
color_dict = {0: 'blue', 1: 'orange', 2: 'red'}
diabetes = {0: 'No Diabetes', 1: 'Pre-Diabetes', 2: 'Diabetes'}

education = {
    x + 1 : 'High School or Less' for x in range(4)
}
education[5] = 'Some College'
education[6] = 'Bachelor\'s or Higher'

age_brackets = {x: f'{15 + 5 * x}-{19 + 5 * x}' for x in range(2, 13)}
age_brackets[1] = '18-24'
age_brackets[13] = '80+'

binary = {0: 'No', 1: 'Yes'}

df['AgeBkt'] = df['Age'].apply(lambda x: age_brackets[int(x)])
df['Diabetes'] = df['Diabetes_012'].apply(lambda x: diabetes[x])
df['Education Level'] = df['Education'].apply(lambda x: education[x])
df['Fruits and Veggies'] = df['Fruits'].astype(int) & df['Veggies'].astype(int)
df['Healthcare'] = df['AnyHealthcare'].apply(lambda x: binary[x])
df['Exercise'] = df['PhysActivity'].apply(lambda x: binary[x])
df['Heavy Drinking'] = df['HvyAlcoholConsump'].apply(lambda x: binary[x])

for col in ['Fruits and Veggies', 'Smoker']:
    df[col] = df[col].apply(lambda x: binary[x])

In [4]:
controllable = ['Smoker', 'Exercise', 'Fruits and Veggies',
                'Heavy Drinking', 'Healthcare', 'Education Level']
output = ['Diabetes']

In [ ]:
# Create Altair selection widgets
dropdowns = [
    alt.binding_select(options=df[feature].unique().tolist(), name=f'{feature}: ')
    for feature in controllable
]

selects = [
    alt.selection_point(
        fields=[feature], bind=dropdowns[i], value=df[feature].unique()[0]
    )
    for i, feature in enumerate(controllable)
]

# Combine all selections
combined_filter = reduce(lambda x, y: x & y, selects)

# Create chart directly from raw data
chart = alt.Chart(df).transform_filter(
    combined_filter
).transform_aggregate(
    count='count()', groupby=['AgeBkt', 'Diabetes']
).transform_joinaggregate(
    total='sum(count)', groupby=['AgeBkt']
).transform_calculate(
    Proportion='datum.count / datum.total'
).mark_bar().encode(
    x=alt.X('AgeBkt:O', title='Age Bucket'),
    y=alt.Y('Proportion:Q', stack='normalize', title='Proportion of Diabetes Diagnosis'),
    color=alt.Color(
        'Diabetes:N', scale=alt.Scale(domain=diabetes.values(), range=color_dict.values()),
        title='Diabetes Classification'),
    tooltip=['AgeBkt:O', 'Diabetes:N', 'Proportion:Q']
).add_params(
    *selects
).properties(
    width=600,
    height=400,
    title='Proportion of Diabetes Levels by Age Bucket'
)

chart.display()


In [6]:
df_raw = df_raw[
    ['Smoker', 'Fruits', 'Veggies', 'AnyHealthcare',
     'HvyAlcoholConsump', 'Education', 'Diabetes_012']
]

df_stratified, _ = train_test_split(
    df_raw,
    stratify=df_raw['Diabetes_012'],
    train_size=1000,
)

In [7]:
def create_matrix(df, output_column='Diabetes_012'):
    features = df.drop(columns=[output_column])
    labels = df[output_column].reset_index(drop=True)

    scaler = StandardScaler()
    features = scaler.fit_transform(features)

    similarity_matrix = cosine_similarity(features)
    np.fill_diagonal(similarity_matrix, -1)

    return similarity_matrix, labels

In [8]:
legend_handles = [
    mpatches.Patch(color='blue', label='No Diabetes'),
    mpatches.Patch(color='orange', label='Pre-diabetes'),
    mpatches.Patch(color='red', label='Diabetes')
]

In [9]:
def create_network(matrix, labels):
    G = nx.Graph()

    for i, status in enumerate(labels):
        G.add_node(i, diabetes=status)

    for i in range(matrix.shape[0]):
        most_similar_idx = np.argmax(matrix[i])
        G.add_edge(i, most_similar_idx, weight=matrix[i, most_similar_idx])

    plt.figure(figsize=(7, 7))
    pos = nx.spring_layout(G)

    colors = [color_dict[int(G.nodes[n]['diabetes'])] for n in G.nodes]

    nx.draw(
        G, pos,
        node_color=colors,
        with_labels=False,
        node_size=5,
        edge_color='grey',
        alpha=0.4
    )

    plt.legend(handles=legend_handles, loc='upper right')

In [ ]:
strat_mat, strat_labels = create_matrix(df_stratified)
create_network(strat_mat, strat_labels)
plt.title('Patient Similarity Network (Stratified) (n=1000)')
plt.show()